In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
table_tmp = np.genfromtxt('../calibrators/Hriz_3sigClip.csv' , delimiter=',', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_drop_riz = table_tmp['PGC']

table_tmp = np.genfromtxt('../calibrators/Hw1_3sigClip.csv' , delimiter=',', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_drop_w1 = table_tmp['PGC']

In [3]:
drop = np.genfromtxt('../calibrators/TF_drop.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_drop = drop['PGC']

faint = np.genfromtxt('../calibrators/TF_faint.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_faint = faint['PGC']

pgc_reject = np.concatenate((pgc_drop, pgc_faint))
pgc_reject = np.unique(pgc_reject)

In [4]:
#################################################################
def Vh2V3k(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    v3k = Vh-25.2*cosl*cosb-245.7*sinl*cosb+276.8*sinb

    return v3k

#################################################################

def Vcmb2Vmod(Vcmb, omegam=0.27):
    
    omegal=1.-omegam
    c=299800.
    z=Vcmb/c
    q0=0.5*(omegam-2.*omegal)
    fmod=1.+0.5*(1.-q0)*z-(1./6)*(1.-q0-3.*q0**2+1.)*z**2
    Vmod=c*z*fmod

    return Vmod
#################################################################

def Vh2Vls(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    vls = Vh-26.*cosl*cosb+317.*sinl*cosb-8.*sinb

    return vls
#################################################################

In [5]:
df = pd.read_csv('all_cf2_tfr.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_allcf2 = df.set_index('PGC')

df_allcf2.loc[5341]['PGC1']  = 5663

df_allcf2['eDM_allcf2'] = df_allcf2.DM * 0. + 0.40
df_allcf2 = df_allcf2.rename(columns={"DM": "DM_allcf2"})
df_allcf2 = df_allcf2.rename(columns={"PGC1": "PGC1_allcf2"})
df_allcf2 = df_allcf2.rename(columns={"Nest": "Nest_allcf2"})
df_allcf2 = df_allcf2.rename(columns={"Vcmb": "Vcmb_allcf2"})

/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [6]:
df = pd.read_csv('all_cf2_tfr.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_sfi = df.set_index('PGC')

df_sfi.loc[5341]['PGC1']  = 5663

df_sfi["eDM_sfi"] = df_sfi.DM * 0. + 0.39
df_sfi = df_sfi.rename(columns={"DMsfi": "DM_sfi"})

df_sfi["DM_sfi"][df_sfi["DM_sfi"]==0] = np.nan
df_sfi["eDM_sfi"][df_sfi["DM_sfi"].isna()] = np.nan
del df_sfi['DM']
df_sfi = df_sfi.rename(columns={"PGC1": "PGC1_sfi"})
df_sfi = df_sfi.rename(columns={"Nest": "Nest_sfi"})
df_sfi = df_sfi.rename(columns={"Vcmb": "Vcmb_sfi"})

/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by Int

In [7]:
df = pd.read_csv('all_cf2_tfr.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_cf2 = df.set_index('PGC')

df_cf2.loc[5341]['PGC1']  = 5663

df_cf2["eDM_cf2"] = df_cf2.DM * 0. + 0.41
df_cf2 = df_cf2.rename(columns={"DMcf2": "DM_cf2"})

df_cf2["DM_cf2"][df_cf2["DM_cf2"]==0] = np.nan
df_cf2["eDM_cf2"][df_cf2["DM_cf2"].isna()] = np.nan
del df_cf2['DM']
df_cf2 = df_cf2.rename(columns={"PGC1": "PGC1_cf2"})
df_cf2 = df_cf2.rename(columns={"Nest": "Nest_cf2"})
df_cf2 = df_cf2.rename(columns={"Vcmb": "Vcmb_cf2"})

/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by Int

In [8]:
df = pd.read_csv('spitzer_cf3_tfr.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_spitzer = df.set_index('PGC')

df_spitzer.loc[5341]['PGC1']  = 5663

ln10 = np.log(10.)
df_spitzer['DM_spitzer']  = (5*np.log10(df_spitzer.Dcf3)+25.)
df_spitzer['eDM_spitzer'] = 5*df_spitzer.eD/df_spitzer.Dcf3/ln10
df_spitzer = df_spitzer.rename(columns={"pgc1": "PGC1_spitzer"})
df_spitzer = df_spitzer.rename(columns={"Nest": "Nest_spitzer"})
df_spitzer = df_spitzer.rename(columns={"Vcmb": "Vcmb_spitzer"})

df_spitzer["eDM_spitzer"] = df_spitzer["DM_spitzer"]*0 + 0.44

df_spitzer["eDM_spitzer"][df_spitzer['C']==1] = 0.40   # color adjusted
df_spitzer["eDM_spitzer"][df_spitzer['C']==0] = 0.50   # otherwise

/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/ehsan/anaconda3/envs/py2/lib/python2.7/site-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/media/Data/Home/anaconda3/envs/py

In [9]:
df = pd.read_csv('2mtf.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_2mtf = df.set_index('PGC')

df_2mtf.loc[5341]['PGC1']  = 5663

df_2mtf = df_2mtf.rename(columns={"DMadj":"DM_2mtf", "eDM":"eDM_2mtf"})
df_2mtf = df_2mtf.rename(columns={"PGC1": "PGC1_2mtf"})
df_2mtf = df_2mtf.rename(columns={"Nest": "Nest_2mtf"})

df_2mtf["eDM_2mtf"] = df_2mtf["DM_2mtf"]*0 + 0.50

df_2mtf['Vls_2mtf'] = Vh2Vls(df_2mtf["glon"], df_2mtf["glat"], df_2mtf["cz2m"])  # using 2mass cz
df_2mtf = df_2mtf.rename(columns={"cz2m": "Vcmb_2mtf"})

df_2mtf = df_2mtf[df_2mtf['Vls_2mtf']>2000]

df_2mtf.head()

/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Name,Dadj,DM_2mtf,eDM_2mtf,2mass,RA,Dec,Vcmb_2mtf,Ktc,Htc,...,Kt,V,Ag,Nest_2mtf,PGC1_2mtf,Ng,Vgm,Sigp,R2t,Vls_2mtf
PGC,,,,,,,,,,,,,,,,,,,,,
70,UGC12900,120.4,35.402,0.5,00005604+2020165,0.23345,20.33799,6804,11.124,11.616,...,11.19,6804,0.34,209949,70,1,6561,122,0.249,7043.671642
76,UGC12901,106.9,35.146,0.5,00005891+2854421,0.24550,28.91172,6899,10.818,11.085,...,10.89,6896,0.21,209247,76,1,6676,139,0.282,7161.795767
80,IC5375,134.8,35.648,0.5,00010478+0432261,0.26987,4.54060,9151,11.029,11.274,...,11.06,9223,0.13,207489,80,1,8992,182,0.371,9334.386184
102,IC5376,84.0,34.621,0.5,00011976+3431326,0.33238,34.52571,5032,10.405,10.704,...,10.47,5029,0.31,210310,102,1,4769,114,0.232,5306.650853
120,UGC12914,98.3,34.963,0.5,00013830+2329011,0.40968,23.48363,4371,9.330,9.591,...,9.39,4349,0.48,202766,120,2,4048,170,0.345,4619.483569


In [10]:
df = pd.read_csv('6dfgs.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_6dfgs = df.set_index('PGC')

ln10 = np.log(10.)
df_6dfgs['DM_6dfgs']  = (5*np.log10(df_6dfgs["D75"])+25.)
df_6dfgs['eDM_6dfgs'] = 5.*df_6dfgs["feD"]/ln10
df_6dfgs = df_6dfgs.rename(columns={"Nest": "Nest_6dfgs"})
df_6dfgs = df_6dfgs.rename(columns={"Vcmb": "Vcmb_6dfgs"})

# df_6dfgs.head()

In [11]:
df = pd.read_csv('flat_tfr.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df_flat = df.set_index('PGC')

df_flat["eDM_flat"] = df_flat.DMadj * 0. + 0.55

df_flat = df_flat.rename(columns={"DMadj":"DM_flat"})
df_flat = df_flat.rename(columns={"PGC1": "PGC1_flat"})
df_flat = df_flat.rename(columns={"Nest": "Nest_flat"})
df_flat = df_flat.rename(columns={"Vcmb": "Vcmb_flat"})

df_flat.head(10)

,DM_flat,DMbig,DMfull,Vh,Vls,Vcmb_flat,Vmod,f,PGC1_flat,Nest_flat,glon,glat,sgl,sgb,DMg,eDMg,eDM_flat
PGC,,,,,,,,,,,,,,,,,
70,35.384,34.817,34.859,6800,7040,6447,6557,1.017,70,209949,107.1780,-40.9837,313.2487,17.7662,35.12,0.12,0.55
642,33.097,32.780,32.572,4441,4718,4130,4175,1.011,642,210102,113.6137,-24.6495,331.3316,18.5421,34.06,0.19,0.55
731,35.444,34.926,34.919,7059,7318,6725,6845,1.018,791,200264,112.2328,-33.0307,322.5668,17.0954,35.23,0.13,0.55
978,33.699,33.290,33.174,5399,5626,5049,5116,1.013,978,207997,110.8971,-43.4288,312.1216,14.2463,34.33,0.16,0.55
1422,34.619,34.141,34.094,5663,5893,5319,5393,1.014,1351,200045,113.7645,-41.7438,314.5450,12.9247,34.59,0.14,0.55
1609,34.076,33.569,33.551,5329,5470,4982,5047,1.013,1609,209583,107.9640,-64.4089,292.2505,6.4025,33.81,0.16,0.55
1751,34.728,34.253,34.203,7640,7756,7301,7441,1.019,1751,0,106.4509,-69.5238,287.5157,4.3315,35.02,0.23,0.55
1797,35.120,34.575,34.595,6350,6605,6030,6126,1.016,1736,200470,117.4270,-31.2540,325.7199,13.4560,34.80,0.14,0.55
2231,34.029,33.552,33.504,5293,5538,4971,5037,1.013,2298,200380,119.2386,-33.6215,323.8154,11.3388,34.09,0.16,0.55


In [12]:
df = pd.read_csv('../calibrators/DMs_EDD.csv.test', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.rename(columns=lambda x: x if x!='DM_w1' else 'DMw1')
df = df.rename(columns=lambda x: x if x!='dDM_w1' else 'dDMw1')
df = df.rename(columns=lambda x: x if x!='pgc' else 'PGC')
df = df.set_index('PGC')

df.loc[69095]['PGC1'] = 69148

## Using hugh quality distances, photoemtry, etc to determine zero points
df_cf4tfr = df[((((df['Sqlt']>2) & ~df.index.isin(pgc_drop_riz)) | 
          ((df['Wqlt']>2) & ~df.index.isin(pgc_drop_w1))) & 
          (~df.index.isin(pgc_reject)) &
          (df['DM_src']>0)
               )]

df_cf4tfr = df_cf4tfr.rename(columns={"pgc1": "PGC1_cf4", 
                                      "Nest": "Nest_cf4",
                                      "DM_best":"DM_cf4", 
                                      "dDM_best":"eDM_cf4"})


df_cf4tfr["eDM_cf4"][df_cf4tfr['DM_src']==1] = 0.37
df_cf4tfr["eDM_cf4"][df_cf4tfr['DM_src']==2] = 0.39

df_cf4tfr = df_cf4tfr.rename(columns={"V3k": "Vcmb_cf4"})

len(df_cf4tfr)

/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


9690

In [13]:
how = 'outer'
df = df_cf4tfr.join(df_allcf2, lsuffix='_cf4', rsuffix='_allcf2', how=how)
df = df.join(df_cf2, lsuffix='_l1', rsuffix='_cf2', how=how)
df = df.join(df_sfi, lsuffix='_l2', rsuffix='_sfi', how=how)
df = df.join(df_spitzer, lsuffix='_l3', rsuffix='_spitzer', how=how)
df = df.join(df_2mtf, lsuffix='_l4', rsuffix='_2mtf', how=how)
df = df.join(df_flat, lsuffix='_l5', rsuffix='_flat', how=how)

# df = df.join(df_6dfgs, lsuffix='_l5', rsuffix='_6dfgs', how=how)


for col in df.columns:
    if col.split('_')[0]=='PGC1' and col!='PGC1_cf4':
        df['PGC1_cf4'] = df['PGC1_cf4'].fillna(df[col])
        
for col in df.columns:
    if col.split('_')[0]=='Nest' and col!='Nest_cf4':
        df['Nest_cf4'] = df['Nest_cf4'].fillna(df[col])

catalogs = ["cf4", "spitzer", "cf2", "sfi", "2mtf", "flat"] # , "allcf2"] #  #, "2mtf"] # , "6dfgs"]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat, 'Nest_'+cat, "DM_"+cat, "eDM_"+cat, "Vcmb_"+cat]

df = df[ss]



print(len(df))


df.head()

12569


,PGC1_cf4,Nest_cf4,DM_cf4,eDM_cf4,Vcmb_cf4,PGC1_spitzer,Nest_spitzer,DM_spitzer,eDM_spitzer,Vcmb_spitzer,...,PGC1_2mtf,Nest_2mtf,DM_2mtf,eDM_2mtf,Vcmb_2mtf,PGC1_flat,Nest_flat,DM_flat,eDM_flat,Vcmb_flat
PGC,,,,,,,,,,,,,,,,,,,,,
2,73150.0,200275.0,34.24,0.39,4726.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,120.0,202766.0,33.29,0.37,4109.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,12.0,210177.0,35.03,0.39,6195.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,16.0,211419.0,34.70,0.37,5312.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,55.0,0.0,34.00,0.37,4454.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
for cat in catalogs:
    df['Vmod_'+cat] = Vcmb2Vmod(df['Vcmb_'+cat])
    logD = (df['DM_'+cat]-25)/5.
    df['logH_'+cat] = np.log10(df['Vmod_'+cat]) - logD 
    
    print cat, 10**df['logH_'+cat][df['Vmod_'+cat]>4000].median()

cf4 74.71127684346841
spitzer 72.70981611169202
cf2 72.21169260070033
sfi 73.01629464566868
2mtf 74.81187678527034
flat 70.09331862587285


/media/Data/Home/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log10
  after removing the cwd from sys.path.


In [15]:
ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
print len(df["PGC1_cf4"][ind])

for cat in catalogs:
    ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
    df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]
    

ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
print len(df["PGC1_cf4"][ind])

ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
df["PGC1_cf4"][ind] = np.nan

69
65


In [16]:
ind = (df["Nest_cf4"].isna()) | (df["Nest_cf4"]==0)
print len(df["Nest_cf4"][ind])

for cat in catalogs:
    ind = (df["Nest_cf4"].isna()) | (df["Nest_cf4"]==0)
    df["Nest_cf4"][ind] = df["Nest_"+cat][ind]
    
ind = (df["Nest_cf4"].isna()) | (df["Nest_cf4"]==0)
df["Nest_cf4"][ind] = np.nan

5406


In [17]:
ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
print len(df["Vmod_cf4"][ind])

for cat in catalogs:
    ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
    df["Vmod_cf4"][ind] = df["Vmod_"+cat][ind]
    
ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
df["Vmod_cf4"][ind] = np.nan

2879


In [18]:
ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
print len(df["Vcmb_cf4"][ind])

for cat in catalogs:
    ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
    df["Vcmb_cf4"][ind] = df["Vcmb_"+cat][ind]
    
ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
df["Vcmb_cf4"][ind] = np.nan

2879


In [19]:
sig3 = pd.read_csv('3SigmaClips.csv', delimiter='|')
sig3 = sig3.rename(columns=lambda x: x.strip())
new_col = {}
for cat in catalogs:
    new_col[cat] = cat+'_flag'

sig3 = sig3.drop(index=0).set_index('PGC')[catalogs]
sig3 = sig3.rename(columns=new_col)
sig3 = sig3.astype('int32')
sig3.index = sig3.index.astype('int32')

In [20]:
sig3.head()

,cf4_flag,spitzer_flag,cf2_flag,sfi_flag,2mtf_flag,flat_flag
PGC,,,,,,
129,0,0,0,0,1,0
181,0,1,0,0,0,0
218,0,0,0,1,0,0
255,0,1,1,0,0,0
701,1,1,1,0,0,0


In [21]:
df.index = df.index.astype('int32')
df = df.join(sig3, lsuffix='_l', rsuffix='_r', how="outer")

for cat in catalogs:
    ind = df[cat+"_flag"].isna()
    df[cat+"_flag"][ind] = 0

df.head()

,PGC1_cf4,Nest_cf4,DM_cf4,eDM_cf4,Vcmb_cf4,PGC1_spitzer,Nest_spitzer,DM_spitzer,eDM_spitzer,Vcmb_spitzer,...,Vmod_2mtf,logH_2mtf,Vmod_flat,logH_flat,cf4_flag,spitzer_flag,cf2_flag,sfi_flag,2mtf_flag,flat_flag
PGC,,,,,,,,,,,,,,,,,,,,,
2,73150.0,200275.0,34.24,0.39,4726.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,120.0,202766.0,33.29,0.37,4109.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
12,12.0,210177.0,35.03,0.39,6195.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
16,16.0,211419.0,34.70,0.37,5312.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
55,55.0,NaN,34.00,0.37,4454.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
for cat in catalogs:
    try:
        df['PGC1_'+cat].loc[5341] = 5663
    except:
        pass
    
    try:
        df['PGC1_'+cat].loc[69095] = 69148
    except:
        pass

In [23]:
myDict = {}
myDict["cf4"] = 0.
myDict["spitzer"] = -0.046
myDict["cf2"] = -0.098
myDict["sfi"] = -0.121
myDict["2mtf"] = -0.054
myDict["flat"] = 0.052


for cat in catalogs:
    df['DM_'+cat] += myDict[cat]
    df['w_'+cat] = (1.-df[cat+'_flag'])/df['eDM_'+cat]**2
    df['xw_'+cat] = (df['DM_'+cat])*df['w_'+cat]

sw = ['w_'+cat for cat in catalogs]
sx = ['xw_'+cat for cat in catalogs]

Err2 = 1./df[sw].sum(axis=1)

df["eDM_av"] = np.sqrt(Err2)
df["DM_av"] = df[sx].sum(axis=1)*Err2   

In [24]:
df['av_flag'] = 0*df['DM_av']
df['av_flag'][df['DM_av'].isna()] = 1

In [25]:
for cat in catalogs:
    df['w_'+cat] = 1./df['eDM_'+cat]**2
    df['xw_'+cat] = df['DM_'+cat]*df['w_'+cat]

Err2 = 1./df[sw].sum(axis=1)
df["DM_av_"] = df[sx].sum(axis=1)*Err2
df["eDM_av_"] = np.sqrt(Err2)

ind = df["av_flag"] == 1
df["eDM_av"][ind] = df["eDM_av_"][ind]
df["DM_av"][ind] = df["DM_av_"][ind]

In [26]:
ss = ['PGC1_cf4', 'Nest_cf4', 'Vcmb_cf4', 'Vmod_cf4', "DM_av", "eDM_av", "av_flag"]
for cat in catalogs:
    ss += ["DM_"+cat, "eDM_"+cat, cat+'_flag']

df = df.fillna(0)

df = df[ss]


df = df.rename(columns={'PGC1_cf4':'PGC1',
                        'Nest_cf4':'Nest',
                        'Vcmb_cf4': 'Vcmb',
                        'Vmod_cf4': 'Vmod'
                       })

In [27]:
df.head(20)

,PGC1,Nest,Vcmb,Vmod,DM_av,eDM_av,av_flag,DM_cf4,eDM_cf4,cf4_flag,...,cf2_flag,DM_sfi,eDM_sfi,sfi_flag,DM_2mtf,eDM_2mtf,2mtf_flag,DM_flat,eDM_flat,flat_flag
PGC,,,,,,,,,,,,,,,,,,,,,
2,73150.0,200275.0,4726.0,4785.113640,34.240000,0.390000,0.0,34.24,0.39,0.0,...,0.0,0.000,0.00,0.0,0.000,0.0,0.0,0.000,0.00,0.0
4,120.0,202766.0,4109.0,4153.715722,33.387534,0.224574,0.0,33.29,0.37,0.0,...,0.0,33.519,0.39,0.0,0.000,0.0,0.0,0.000,0.00,0.0
12,12.0,210177.0,6195.0,6296.413824,35.030000,0.390000,0.0,35.03,0.39,0.0,...,0.0,0.000,0.00,0.0,0.000,0.0,0.0,0.000,0.00,0.0
16,16.0,211419.0,5312.0,5386.635077,34.700000,0.370000,0.0,34.70,0.37,0.0,...,0.0,0.000,0.00,0.0,0.000,0.0,0.0,0.000,0.00,0.0
55,55.0,0.0,4454.0,4506.520342,34.177904,0.224574,0.0,34.00,0.37,0.0,...,0.0,34.399,0.39,0.0,0.000,0.0,0.0,0.000,0.00,0.0
68,68.0,0.0,7338.0,7480.113602,34.810000,0.370000,0.0,34.81,0.37,0.0,...,0.0,0.000,0.00,0.0,0.000,0.0,0.0,0.000,0.00,0.0
70,70.0,209949.0,6447.0,6556.802476,35.252999,0.191975,0.0,35.12,0.37,0.0,...,0.0,35.279,0.39,0.0,35.348,0.5,0.0,35.436,0.55,0.0
76,76.0,209247.0,6583.0,6697.467170,34.827127,0.204859,0.0,34.73,0.37,0.0,...,0.0,34.679,0.39,0.0,35.092,0.5,0.0,0.000,0.00,0.0
80,80.0,207489.0,9151.0,9371.581197,35.594000,0.500000,0.0,0.00,0.00,0.0,...,0.0,0.000,0.00,0.0,35.594,0.5,0.0,0.000,0.00,0.0


In [28]:
df.to_csv("TFR_CF4_components_bar.csv", sep='|')

In [29]:
table   = np.genfromtxt('TFR_CF4_components_bar.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)

colnames = table.dtype.names

## table is a structured array
myTable = {}
for name in table.dtype.names:
    myTable[name] = table[name]
table = myTable
## table is now a dictionary

myTable = Table()

for key in colnames:
    if key in ["PGC", "PGC1", "Nest"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int))) 
    elif key in ["Vcmb", "Vmod"]+["av_flag","cf4_flag","spitzer_flag","cf2_flag","sfi_flag","2mtf_flag","flat_flag"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int)))
    else:
        myTable.add_column(Column(data=table[key], name=key, format='%0.2f'))

## to be used on EDD
myTable.write('TFR_CF4_components_bar', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True)

In [30]:
df.loc[5341]["PGC1"]

5663.0

In [31]:
df.loc[69095]["PGC1"]

69148.0